## Pipeline обработки запроса

### Загрузка данных и функций (достаточно выполнить 1 раз)

In [1]:
from main import setup_environment, load_data
from retriever import retriever
from planner import planner
from dreamer import dreamer
from grounder import grounder
from executor import executor
from schemas import RetrieverOut, PlannerOut, DreamerOut

from openai import OpenAI
from config import PipelineConfig


api_key, db_path = setup_environment()
db = load_data(db_path, wave_filter=["2025-03"])    # Фильтр по последней волне

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

2025-11-13 18:15:33 | INFO     | main                 | Environment loaded: DB_PATH=C:\Users\kateu\Documents\Ivanovs Index\V2\DB\25W3\db_CORRECTED.parquet
2025-11-13 18:15:33 | INFO     | main                 | Loading data from C:\Users\kateu\Documents\Ivanovs Index\V2\DB\25W3\db_CORRECTED.parquet
2025-11-13 18:15:34 | INFO     | main                 | Full dataset shape: (26505646, 5)
2025-11-13 18:15:34 | INFO     | main                 | Filtered by waves: ['2025-03'], new shape: (672405, 5)


### Настройка используемых моделей и их температур

Если нужно поменять модель, это делается здесь

In [2]:
PPL_cfg = PipelineConfig.setup(
    df=db, client=client,
    # параметры ретривера
    retriever_params={
        # "model": "openrouter/polaris-alpha",
        "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        "temperature": 0.5,
        "reasoning_effort": "high"
    },
    # параметры аналитика-выдумщика
    dreamer_params={
        # "model": "openrouter/polaris-alpha",
        "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        "temperature": 1,
        "reasoning_effort": "high"
    },
    # параметры планировщика
    planner_params={
        # "model": "deepseek/deepseek-chat-v3.1",
        # !!! "model": "deepseek/deepseek-chat-v3.1", # Платная модель
        "temperature": 0.4
    }
)

### Еще варианты моделей
# "alibaba/tongyi-deepresearch-30b-a3b:free"
# "meta-llama/llama-4-maverick:free"
# !!! "deepseek/deepseek-chat-v3.1" # Платная модель

### Retriever

Извлекает релевантные вопросы на основе запроса пользователя и всего набора вопросов

Пользовательский запрос

In [3]:
# user_query = "Я хочу посчитать индекс потребительской уверенности по потребителям из москвы"
# user_query = "Я хочу посчитать размер средних сбережений и норму сбережений среди тех, у кого они есть"
user_query = "Я хочу посчитать долю взаимопроникновения клиентов Чижика и Пятерочки"
# user_query = "Мне нужны все вопросы, связанные с автотранспортом (сроки владения, предпочтения по маркам, планы по покупке и пр.)"
# user_query = "Мне нужны все вопросы, связанные с автотранспортом (сроки владения, предпочтения по маркам, планы по покупке и пр.)"

Обращение к LLM

> ! Возможны проблемы парсинга ответов

In [ ]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

# retriever_out = retriever(user_query, PPL_cfg)
# retriever_out.save("cur_retrieved.json")

2025-11-13 18:15:41 | INFO     | retriever            | Starting retriever for query: Я хочу посчитать долю взаимопроникновения клиентов Чижика и Пятерочки...
2025-11-13 18:15:41 | DEBUG    | retriever            | 1/2 part of questions...
2025-11-13 18:15:41 | DEBUG    | retriever            | Prompt length: 86959
2025-11-13 18:15:44 | INFO     | httpx                | HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-13 18:15:55 | DEBUG    | retriever            | 2/2 part of questions...
2025-11-13 18:15:55 | DEBUG    | retriever            | Prompt length: 92065
2025-11-13 18:15:58 | INFO     | httpx                | HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-13 18:16:15 | DEBUG    | retriever            | Parsing retriever response
2025-11-13 18:16:15 | DEBUG    | retriever            | Parsed question: '[A3] В каких магазинах из перечисленных ниже Вы заказывали товары онлайн (за последние 6 м

Можно посмотреть reasoning, если модель его поддерживает и он был указан в параметрах

In [7]:
# print(retriever_out.reasoning)

In [5]:
retriever_out = RetrieverOut.load("cur_retrieved.json")
print(retriever_out)

1. '[A3] В каких магазинах из перечисленных ниже Вы заказывали товары онлайн (за последние 6 месяцев)?'
	Reason: Позволяет выявить, какие магазины, включая "Чижик" и "Пятерочка", использовались для онлайн-покупок. Это важно для понимания цифровой активности клиентов.
2. '[A3_main] В каких магазинах из перечисленных ниже Вы совершили больше всего покупок за последние 6 месяцев (в денежном выражении) ?'
	Reason: Помогает определить, какие магазины являются основными для клиентов по сумме трат, включая "Чижик" и "Пятерочка". Это ключевой показатель для анализа пересечения аудитории.
3. '[C6_offline] Как часто Вы совершали покупки в этих магазинах в последнем месяце? @ Пятерочка'
	Reason: Фиксирует частоту офлайн-покупок в "Пятерочке", что необходимо для расчета доли клиентов, посещающих оба магазина.
4. '[C6_offline] Как часто Вы совершали покупки в этих магазинах в последнем месяце? @ Чижик'
	Reason: Аналогично предыдущему пункту, но для "Чижика". Вместе с данными по "Пятерочке" позволяе

### Dreamer

Строит план анализа без привязки к функционалу

In [ ]:
# dreamer_out = dreamer(user_query, PPL_cfg)
# dreamer_out.save("cur_dreamed.json")

2025-11-13 18:17:12 | INFO     | httpx                | HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
dreamer_out = DreamerOut.load("cur_dreamed.json")
print(dreamer_out)

####################
АНАЛИЗ
####################


**План анализа для расчёта доли взаимопроникновения клиентов Чижика и Пятерочки:**

1. **Определение ключевых переменных:**
   - Проверить, разрабатывался ли опросник с поддержкой множественного выбора в вопросах (например, Q115). Если **да**, то использовать данные этого вопроса.
   - Если множественный выбор не предусмотрен, использовать данные из **C6_offline** для Чижика и Пятерочки, предполагая, что вопросы заданы одному респонденту, если логика опросника предполагает связанные ответы.

2. **Обработка данных:**
   - **Вариант A (множественный выбор в Q115):**
     - Отобрать респондентов, которые выбрали и Пятерочку, и Чижика в вопросе Q115.
     - Подсчитать количество таких респондентов (A).
     - Разделить на общее число опрошенных (N): **Dоля = A / N**.
   - **Вариант B (C6_offline):**
     - Объединить ответы по C6_offline для Чижика и Пятерочки, если они относятся к полю getAllContent.
     - Подсчитать респондентов, давших

Можно отдельно посмотреть **анализ** и **reasoning**, раскомментировав соответствующую строку

In [11]:
# print(dreamer_out.analysis)
# print(dreamer_out.reasoning)

### Planner

Строит план на основе пользовательского запроса и набора релевантных ответов, к которым подмешиваются их ответы

В план включаются команды из [`capability_spec.py`](./capability_spec.py)

In [12]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

planner_out = planner(user_query, dreamer_out, PPL_cfg)
planner_out.save("cur_planned.json")

2025-11-13 18:18:11 | INFO     | planner              | Starting planner for query: Я хочу посчитать долю взаимопроникновения клиентов Чижика и Пятерочки...
2025-11-13 18:18:11 | DEBUG    | planner              | Generated prompt of length: 8758
2025-11-13 18:18:11 | DEBUG    | planner              | Calling LLM with model: meta-llama/llama-3.3-70b-instruct:free
2025-11-13 18:18:13 | INFO     | httpx                | HTTP Request: POST https://openrouter.ai/api/v1/responses "HTTP/1.1 200 OK"
2025-11-13 18:18:29 | DEBUG    | planner              | Plan received: 5 steps
2025-11-13 18:18:29 | DEBUG    | planner              | Plan analysis: План анализа для расчета доли взаимопроникновения клиентов Чижика и Пятерочки включает в себя определение ключевых переменных, обработку данных, корректировку под выборку и дополнительные проверки.
2025-11-13 18:18:29 | INFO     | planner              | Planner completed: 5 steps generated
2025-11-13 18:18:29 | DEBUG    | planner              | Planne

In [13]:
planner_out = PlannerOut.load("cur_planned.json")
print(planner_out)

АНАЛИЗ: План анализа для расчета доли взаимопроникновения клиентов Чижика и Пятерочки включает в себя определение ключевых переменных, обработку данных, корректировку под выборку и дополнительные проверки.
0. [s1] OperationType.LOAD_DATA
	Goal: Загрузить данные результатов опросов
	Inputs: {'waves': ['2025-03']}
	Outputs: ['dataset']
	Constraints: {}
	Depends on: []
1. [s2] OperationType.FILTER
	Goal: Отфильтровать респондентов, которые выбрали и Пятерочку, и Чижика в вопросе Q115
	Inputs: {'dataset': 'dataset', 'question': '[Q115] В каких магазинах Вы делаете покупки?', 'answer_values': ['Пятерочка', 'Чижик'], 'logic': 'include'}
	Outputs: ['filtered_dataset']
	Constraints: {}
	Depends on: ['s1']
2. [s3] OperationType.PIVOT
	Goal: Подсчитать количество респондентов, которые выбрали и Пятерочку, и Чижика
	Inputs: {'dataset': 'filtered_dataset', 'question': '[Q115] В каких магазинах Вы делаете покупки?'}
	Outputs: ['pivot']
	Constraints: {}
	Depends on: ['s2']
3. [s4] OperationType.LOAD

### Grounder

Привязка шагов плана к имеющимся функциям (из [`operations.py`](./operations.py))

In [14]:
grounder_out = grounder(planner_out)

2025-11-13 18:19:19 | INFO     | grounder             | Starting grounder
2025-11-13 18:19:19 | DEBUG    | grounder             | Grounding step 1/5: s1
2025-11-13 18:19:19 | DEBUG    | grounder             | Successfully grounded step s1: LOAD_DATA
2025-11-13 18:19:19 | DEBUG    | grounder             | Grounding step 2/5: s2
2025-11-13 18:19:19 | DEBUG    | grounder             | Successfully grounded step s2: FILTER
2025-11-13 18:19:19 | DEBUG    | grounder             | Grounding step 3/5: s3
2025-11-13 18:19:19 | DEBUG    | grounder             | Successfully grounded step s3: PIVOT
2025-11-13 18:19:19 | DEBUG    | grounder             | Grounding step 4/5: s4
2025-11-13 18:19:19 | DEBUG    | grounder             | Successfully grounded step s4: LOAD_DATA
2025-11-13 18:19:19 | DEBUG    | grounder             | Grounding step 5/5: s5
2025-11-13 18:19:19 | DEBUG    | grounder             | Successfully grounded step s5: PIVOT
2025-11-13 18:19:19 | INFO     | grounder             | G

### Executor

Валидирует план, осуществляет топологическую сортировку, выполняет последовательность шагов

In [15]:
ctx = {"dataset": db}
ctx = executor(grounder_out, ctx)

2025-11-13 18:19:22 | INFO     | executor             | Starting executor with 5 steps
2025-11-13 18:19:22 | DEBUG    | executor             | Validating step dependencies
2025-11-13 18:19:22 | DEBUG    | executor             | Performing topological sort
2025-11-13 18:19:22 | DEBUG    | executor             | Topological sort completed: ['s1', 's2', 's3', 's4', 's5']
2025-11-13 18:19:22 | DEBUG    | executor             | Dependencies validation passed
2025-11-13 18:19:22 | DEBUG    | executor             | Performing topological sort
2025-11-13 18:19:22 | DEBUG    | executor             | Topological sort completed: ['s1', 's2', 's3', 's4', 's5']
2025-11-13 18:19:22 | INFO     | executor             | Steps execution order: ['s1', 's2', 's3', 's4', 's5']
2025-11-13 18:19:22 | INFO     | executor             | Executing step 1/5: s1 (LOAD_DATA)
2025-11-13 18:19:22 | DEBUG    | executor             |   waves = ['2025-03'] (literal)
2025-11-13 18:19:22 | DEBUG    | executor             

Далее можно извлекать созданные таблицы из `ctx`

In [16]:
opts = [f"{i}. '{n}'" for i, n in enumerate(ctx.keys(), start=1)]
opts = "\n".join(opts)

print(f"Доступные варианты:\n{opts}")

Доступные варианты:
1. 'dataset'
2. 'filtered_dataset'
3. 'pivot'
4. 'total_respondents'
5. 'final_pivot'


Пока что извлекать и смотреть можно только ручками...

In [17]:
ctx["final_pivot"]

wave,2025-03
answer,
Fix Price,1061
O'Кей,173
SPAR,211
Азбука Вкуса,66
Ашан,349
Бристоль,297
В1 (Первый выбор),26
Верный,133
Виктория,41
